<a href="https://colab.research.google.com/github/asheta66/Machine-Learning-2022/blob/main/ML_COPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Importing the libraries
import math
import numpy as np
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set_style("whitegrid")

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Plot settings

In [13]:
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 11
plt.rcParams['xtick.major.size'] = 10
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['ytick.major.size'] = 10
plt.rcParams['ytick.major.width'] = 1
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.minor.size'] = 5
plt.rcParams['ytick.minor.width'] = 1
plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['figure.figsize'] = 7,4
sns.set_style('ticks')

# Reading the data



In [14]:
# Importing the datasets
df = pd.read_csv('COPD_cleaned_data_Alaa.csv')

# Display the data

In [15]:
df.iloc[:,:-1].describe()

,height,weight,age,sex,race,pre-fvc,post-fvc,fev1-pre,fev1-post,fev1-fvc-pre-actual,fev1-fvc post actual,tlc-pred,dlco-pred,fev1-fev6-pre-pred,fev1-fev6-post-pred,smoker,cough,d1sp0ea
count,1603.000000,1603.000000,1603.000000,1603.000000,1603.00000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000,1603.000000
mean,64.548097,193.662913,66.944479,0.524017,0.40549,78.180911,79.539183,73.278228,75.637679,72.750468,74.021834,88.016220,73.148472,92.976294,94.449158,0.631316,0.581410,0.772302
std,4.054318,53.617939,12.033507,0.499579,0.49114,19.825795,19.484328,23.039914,23.843730,14.574022,15.115870,34.485731,25.046332,15.381318,16.189549,0.482599,0.493482,0.419478
min,6.500000,58.750000,18.000000,0.000000,0.00000,8.000000,1.310000,15.000000,0.000000,23.000000,21.000000,11.000000,14.000000,40.000000,9.000000,0.000000,0.000000,0.000000
25%,62.000000,155.000000,61.000000,0.000000,0.00000,64.000000,66.000000,56.500000,60.000000,66.000000,66.000000,74.000000,55.000000,85.000000,85.000000,0.000000,0.000000,1.000000
50%,64.250000,188.000000,69.000000,1.000000,0.00000,77.000000,80.000000,74.000000,76.000000,77.000000,78.000000,87.000000,72.000000,97.000000,99.000000,1.000000,1.000000,1.000000
75%,67.250000,224.000000,75.000000,1.000000,1.00000,92.000000,93.000000,90.000000,92.000000,83.000000,85.000000,99.000000,89.000000,104.000000,106.000000,1.000000,1.000000,1.000000
max,82.000000,432.000000,95.000000,1.000000,1.00000,139.000000,143.000000,143.000000,354.000000,100.000000,108.000000,996.000000,198.000000,126.000000,126.000000,1.000000,1.000000,1.000000


In [16]:
df.head(10)

,height,weight,age,sex,race,pre-fvc,post-fvc,fev1-pre,fev1-post,fev1-post-change,...,fev1-fvc post actual,tlc-pred,dlco-pred,fev1-fev6-pre-pred,fev1-fev6-post-pred,fev1-fev6-post-change,smoker,cough,d1sp0ea,COPD
0,67.00,329.0,55,0,0,57,60.0,59,65.0,9+,...,86,77,106,102,106,4+,0,1,1,1
1,68.75,204.0,66,0,0,114,107.0,78,72.0,7-,...,52,128,107,69,68,1-,1,0,0,1
2,68.00,197.0,59,0,1,85,86.0,87,89.0,2+,...,80,94,90,99,100,0+,0,1,1,1
3,62.00,141.0,79,1,0,80,91.0,83,82.0,1-,...,69,93,77,102,89,11-,1,0,1,1
4,61.50,159.0,80,1,0,84,84.0,77,77.0,0+,...,70,96,70,90,89,0+,1,1,0,1
5,63.00,281.0,66,0,1,64,66.0,71,72.0,2+,...,85,78,93,108,108,0+,0,0,1,1
6,66.50,158.0,62,1,0,117,115.0,100,101.0,1+,...,68,129,113,86,87,0+,1,1,1,1
7,71.50,202.0,72,0,0,58,56.0,52,51.0,0+,...,69,54,46,90,90,0+,1,0,1,1
8,51.25,158.0,85,1,1,110,109.0,115,116.0,0+,...,83,112,158,106,108,1+,0,0,1,1
9,69.00,310.0,74,0,0,48,60.0,39,57.0,45+,...,72,84,77,80,94,17+,0,0,1,1


**Data Visulatization**

In [17]:
from matplotlib import pyplot as plt

# fig, (ax1, ax2,ax3) = plt.subplots(1,3)
# fig.suptitle('Data Statistics')
# ax1.plot(OSA, 'o-')
# ax1.set_ylabel('Damped oscillation')
# ax2.plot(x2, y2, '.-')
# ax2.set_xlabel('time (s)')
# ax2.set_ylabel('Undamped')
# plt.show()

# plt.subplot(13)
# # equivalent but more general
# ax1 = plt.subplot(2, 2, 1)
# # add a subplot with no frame
# ax2 = plt.subplot(222, frameon=False)
# # add a polar subplot
# plt.subplot(223, projection='polar')

# snore_freq	snore_vol	choke_freq	apnea_freq	awake_freq	apnea

# fig, axes = plt.subplots(1,7,figsize=(14,5),sharey=True)
# sns.countplot(ax=axes[0],data=df, x='apnea')
# sns.countplot(ax=axes[1],data=df, x='snore_freq')
# sns.countplot(ax=axes[2],data=df, x='snore_vol')
# sns.countplot(ax=axes[3],data=df, x='choke_freq')
# sns.countplot(ax=axes[4],data=df, x='apnea_freq')
# sns.countplot(ax=axes[5],data=df, x='snore_freq')
# sns.countplot(ax=axes[6],data=df, x='awake_freq')
# plt.savefig('Statistics.png')


# Normalizing the data

In [ ]:
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler
 
x_data = df.drop('COPD', axis = 1)
Y      = df['COPD']

sc = StandardScaler()
X  = sc.fit_transform(x_data)


# Showing the shape of the data: rows and columns
# [n, m] = df.shape
# n, m
# x_data = df.iloc[:,0:m-1]
# y      = df.iloc[:,m-1]

# # Normalize data
# x = (x_data - np.min(x_data))/(np.max(x_data) - np.min(x_data)).values
# x


# Preparing data for machine learning

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.1,random_state=0)

## Correlation between different variables

In [ ]:
# Correlation heatmap with mask for upper traingle
# Correlation between different variables
corr = df.corr()
# Set up the matplotlib plot configuration
f, ax = plt.subplots(figsize=(12, 6))
# Generate a mask for upper traingle
mask = np.triu(np.ones_like(corr, dtype=bool))
# Configure a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)
# Draw the heatmap
sns.heatmap(corr, annot=True, mask = mask, cmap=cmap)
plt.savefig('Heatmap.png')


## Class distribution in OSA dataset

In [ ]:
fig, axs = plt.subplots(figsize=(12, 3), tight_layout=True)
sns.boxplot(data=df)
plt.savefig("BoxplotData.png")
plt.show()

In [ ]:
# snore_freq	snore_vol	choke_freq	apnea_freq	awake_freq	apnea
# sns.pairplot(df, vars=["choke_freq","apnea_freq","awake_freq","apnea"])
# plt.grid()
# plt.savefig('Correction with Apnea.png')
# plt.show()

In [ ]:
def calculate_confusion(name,y_train, y_pred_train, y_test, y_pred_test):
  
  # Making the Confusion Matrix 

  from sklearn.metrics import confusion_matrix
  from sklearn.metrics import plot_confusion_matrix

  cm1  = confusion_matrix(y_train, y_pred_train)
  cm2  = confusion_matrix(y_test, y_pred_test)

  test_calc = pd.concat([pd.DataFrame(y_train).reset_index(drop=True),pd.DataFrame(y_pred_train).reset_index(drop=True)],axis=1)
  test_calc.rename(columns={0: 'predicted'}, inplace=True)

  Accuracy1  = (cm1[0,0] + cm1[1,1]) / (cm1[0,0] + cm1[0,1] + cm1[1,0] + cm1[1,1])
  precision1 = cm1[1,1] / (cm1[1,1] + cm1[0,1])
  recall1    = cm1[1,1] / (cm1[1,1] + cm1[1,0])

  print('Performance Criteria in Training Case for %s' %name)
  print('Accuracy  %.3f' %Accuracy1)
  print ('precision %.3f' % precision1)
  print('recall %.3f' % recall1)

  ##########################################
  test_calc = pd.concat([pd.DataFrame(y_train).reset_index(drop=True),pd.DataFrame(y_pred_train).reset_index(drop=True)],axis=1)
  test_calc.rename(columns={0: 'predicted'}, inplace=True)

  Accuracy2  = (cm2[0,0] + cm2[1,1]) / (cm2[0,0] + cm2[0,1] + cm2[1,0] + cm2[1,1])
  precision2 = cm2[1,1] / (cm2[1,1] + cm2[0,1])
  recall2    = cm2[1,1] / (cm2[1,1] + cm2[1,0])

  print('====================')
  print('Performance Criteria in Testing Case for %s' %name)
  print('Accuracy  %.3f' %Accuracy2)
  print ('precision %.3f' % precision2)
  print('recall %.3f' % recall2)

  cm_train = pd.DataFrame(cm1, range(2), range(2))
  cm_test  = pd.DataFrame(cm2, range(2), range(2))

  f,(ax1,ax2) = plt.subplots(1,2,figsize=(7, 3), sharey=True)

  g1 = sns.heatmap(cm_train,cbar=False,ax=ax1,annot=True, fmt=".1f",cmap="viridis",annot_kws={"size": 15})
  g1.set_title('Training case',fontname= 'serif',fontsize=10, color='k')

  g2 = sns.heatmap(cm_test,cbar=False,ax=ax2,annot=True, fmt=".1f",cmap="viridis",annot_kws={"size": 15})
  g2.set_title('Testing case', fontname= 'serif',fontsize=10, color='k')
  
  f.suptitle(' Confusion matrix for '+name, fontsize=11)
  plt.savefig(name + ".png")
  plt.show()

In [ ]:
#set up plotting area
from matplotlib.pyplot import figure
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

# Making the Confusion Matrix 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

# We now prepare our selected models
models  = []
results = []
names   = []
scoring = 'accuracy'
names   = ['LRC', 'SVC', 'GBC', 'GNB', 'RFC', 'KNC']

#LogisticRegression ####################################
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, y_train)
y_pred_train = lr.predict(x_train)
y_pred_test  = lr.predict(x_test)
calculate_confusion("LR",y_train, y_pred_train, y_test, y_pred_test)
fpr1, tpr1, _ = metrics.roc_curve(y_test, y_pred_test)
auc = round(metrics.roc_auc_score(y_test, y_pred_test), 4)
cv_results = cross_val_score(lr, x_train, y_train)
results.append(cv_results)

#SVM ####################################
from sklearn.svm import SVC
svm = SVC()
svm.fit(x_train, y_train)
y_pred_train = svm.predict(x_train)
y_pred_test  = svm.predict(x_test)
calculate_confusion("SVM",y_train, y_pred_train, y_test, y_pred_test)
# plt.savefig('ConfMatrix_SVC.png')
fpr2, tpr2, _ = metrics.roc_curve(y_test, y_pred_test)
auc = round(metrics.roc_auc_score(y_test, y_pred_test), 4)
cv_results = cross_val_score(svm, x_train, y_train)
results.append(cv_results)
# ax1.plot(fpr2,tpr2,label="SVC, AUC="+str(auc))

#GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
GBC = GradientBoostingClassifier()
GBC.fit(x_train, y_train)
y_pred_train = GBC.predict(x_train)
y_pred_test  = GBC.predict(x_test)
calculate_confusion("GBC",y_train, y_pred_train, y_test, y_pred_test)
# plt.savefig('ConfMatrix_GBC.png')
fpr3, tpr3, _ = metrics.roc_curve(y_test, y_pred_test)
auc = round(metrics.roc_auc_score(y_test, y_pred_test), 4)
cv_results = cross_val_score(GBC, x_train, y_train)
results.append(cv_results)
# ax1.plot(fpr3,tpr3,label="GBC, AUC="+str(auc))

#GaussianNB
from sklearn.naive_bayes import GaussianNB
GNB = GaussianNB()
GNB.fit(x_train, y_train)
y_pred_train = GNB.predict(x_train)
y_pred_test  = GNB.predict(x_test)
calculate_confusion("GNB",y_train, y_pred_train, y_test, y_pred_test)
# plt.savefig('ConfMatrix_GNB.png')
fpr4, tpr4, _ = metrics.roc_curve(y_test, y_pred_test)
auc = round(metrics.roc_auc_score(y_test, y_pred_test), 4)
cv_results = cross_val_score(GNB, x_train, y_train)
results.append(cv_results)
# ax1.plot(fpr4,tpr4,label="GaussianNB, AUC="+str(auc))

# A random forest classifier:
# A random forest is a meta estimator that fits a number of decision tree 
# classifiers on various sub-samples of the dataset and uses averaging to 
# improve the predictive accuracy and control over-fitting. The sub-sample size 
# is controlled with the max_samples parameter if bootstrap=True (default), 
# otherwise the whole dataset is used to build each tree.

# KNN Model
from sklearn.neighbors import KNeighborsClassifier
KNC = KNeighborsClassifier()
KNC.fit(x_train, y_train)
y_pred_train = KNC.predict(x_train)
y_pred_test  = KNC.predict(x_test)
calculate_confusion("KNC",y_train, y_pred_train, y_test, y_pred_test)
# plt.savefig('ConfMatrix_KNC.png')
fpr5, tpr5, _ = metrics.roc_curve(y_test, y_pred_test)
auc = round(metrics.roc_auc_score(y_test, y_pred_test), 4)
cv_results = cross_val_score(KNC, x_train, y_train)
results.append(cv_results)
# ax1.plot(fpr5,tpr5,label="KNN, AUC="+str(auc))

# Decision Tree Model
from sklearn import tree
DT = tree.DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None)
DT.fit(x_train, y_train)
y_pred_train = DT.predict(x_train)
y_pred_test  = DT.predict(x_test)
calculate_confusion("DT",y_train, y_pred_train, y_test, y_pred_test)
# plt.savefig('ConfMatrix_KNC.png')
fpr6, tpr6, _ = metrics.roc_curve(y_test, y_pred_test)
auc = round(metrics.roc_auc_score(y_test, y_pred_test), 4)
cv_results = cross_val_score(DT, x_train, y_train)
results.append(cv_results)
# tree.plot_tree(clf)
# ax1.plot(fpr6,tpr6,label="DT, AUC="+str(auc))

#RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None)
RFC.fit(x_train, y_train)
y_pred_train = RFC.predict(x_train)
y_pred_test  = RFC.predict(x_test)
calculate_confusion("RFC",y_train, y_pred_train, y_test, y_pred_test)
# plt.savefig('ConfMatrix_RFC.png')
fpr7, tpr7, _ = metrics.roc_curve(y_test, y_pred_test)
auc = round(metrics.roc_auc_score(y_test, y_pred_test), 4)
cv_results = cross_val_score(RFC, x_train, y_train)
results.append(cv_results)
# ax1.plot(fpr7,tpr7,label="RandomForestClassifier, AUC="+str(auc))

#Artificial Neural Network
from sklearn.neural_network import MLPClassifier
ANN = MLPClassifier(solver='adam', hidden_layer_sizes=(15, 1), learning_rate='adaptive',
                    random_state=1, max_iter=5000)
ANN.fit(x_train, y_train)
y_pred_train = ANN.predict(x_train)
y_pred_test  = ANN.predict(x_test)
calculate_confusion("ANN",y_train, y_pred_train, y_test, y_pred_test)
# plt.savefig('ConfMatrix_RFC.png')
fpr8, tpr8, _ = metrics.roc_curve(y_test, y_pred_test)
auc = round(metrics.roc_auc_score(y_test, y_pred_test), 4)
cv_results = cross_val_score(ANN, x_train, y_train)
results.append(cv_results)
# ax1.plot(fpr8,tpr8,label="MLPClassifier, AUC="+str(auc))

In [ ]:
# import graphviz
# import pydotplus

# names = ['LRC', 'SVC', 'GBC', 'GNB', 'KNC', 'DT','RFC', 'ANN']

# fig = plt.figure(figsize=(25,20))
# dot_data = tree.export_graphviz(DT, out_file=None,  feature_names=features,  class_names=names,  filled=True, rounded=True,  special_characters=True)  
# graph = graphviz.Source(dot_data)  
# graph 

# pydot_graph = pydotplus.graph_from_dot_data(dot_data)
# pydot_graph.write_png('original_tree.png')
# pydot_graph.set_size('"5,5!"')
# pydot_graph.write_png('resized_tree.png')
# gvz_graph = graphviz.Source(pydot_graph.to_string())
# graph.render(view=True)

In [ ]:
# fig = plt.figure(figsize=(25,20))
# _    = tree.plot_tree(DT, 
#                    feature_names=features,  
#                    class_names=names,
#                    filled=True)

In [ ]:
# import matplotlib.pyplot as plt

# plt.title('Receiver Operating Characteristic')
# plt.plot(fpr1, tpr1, label="LogisticRegression, AUC="+str(auc))
# plt.plot(fpr2, tpr2, label="SVC, AUC="+str(auc))
# plt.legend(loc = 'lower right')
# plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0, 1])
# plt.ylim([0, 1])
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# ax = plt.axes()
# fig = plt.figure(figsize=(8,6))
# ax.plot(fpr1,tpr1,label="LogisticRegression, AUC="+str(auc))
# ax.plot(fpr2,tpr2,label="SVC, AUC="+str(auc))
# ax.plot(fpr3,tpr3,label="GBC, AUC="+str(auc))
# ax.plot(fpr4,tpr4,label="GaussianNB, AUC="+str(auc))
# ax.plot(fpr5,tpr5,label="RandomForestClassifier, AUC="+str(auc))
# ax.plot(fpr6,tpr6,label="KNN, AUC="+str(auc))
# # plt.legend()
# plt.savefig('ROC.png')

In [ ]:
# fig = plt.figure(figsize=(8,4))
# fig.suptitle('ROC')
# ax = fig.add_subplot(111)
# plt.plot(fpr1,tpr1,label="LogisticRegression, AUC="+str(auc))
# plt.plot(fpr2,tpr2,label="SVC, AUC="+str(auc))
# ax.set_xticklabels(names)
# plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(6,6))

# Add the models to the list that you want to view on the ROC plot
models = [
{
    'label': 'Logistic Regression',
    'model': LogisticRegression(),
},
{
    'label': 'SVM',
    'model': SVC(gamma='auto', probability=True),
},
{
    'label': 'Gradient Boosting',
    'model': GradientBoostingClassifier(),
},
{
    'label': 'GaussianNB',
    'model': GaussianNB(),
},
{
    'label': 'KNeighborsClassifier',
    'model': KNeighborsClassifier(),
},
{
    'label': 'DecisionTreeClassifier',
    'model': tree.DecisionTreeClassifier(),
},
{
    'label': 'RandomForestClassifier',
    'model': RandomForestClassifier(),
},
{
    'label': 'MLPClassifier',
    'model': MLPClassifier(),
}
]

# Below for loop iterates through your models list
for m in models:
    model = m['model'] # select the model
    model.fit(x_train, y_train) # train the model
    y_pred=model.predict(x_test) # predict the test data

# Compute False postive rate, and True positive rate
    fpr, tpr, thresholds = metrics.roc_curve(y_test, model.predict_proba(x_test)[:,1])

# Calculate Area under the curve to display on the plot
    auc = metrics.roc_auc_score(y_test,model.predict(x_test))

# Now, plot the computed values
    plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % (m['label'], auc))

# Custom settings for the plot 
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('ROC_Curves.png')
plt.show()   # Display


In [ ]:
# from sklearn.metrics import RocCurveDisplay

# fig = plt.figure(figsize=(8,4))
# fig.suptitle('ROC')

# disp = RocCurveDisplay.from_estimator(lr, x_test, y_test, marker="+")
# _ = disp.ax_.set_title("Receiver Operating Characteristic curve")
# plt.xlabel("False positive rate")
# plt.ylabel("True positive rate \n(also known as sensitivity or recall)")
# plt.xlim(0, 1)
# plt.ylim(0, 1)
# plt.legend(loc="lower right")

# disp = RocCurveDisplay.from_estimator(svm, x_test, y_test, marker="+")
# _ = disp.ax_.set_title("Receiver Operating Characteristic curve")

# plt.xlabel("False positive rate")
# plt.ylabel("True positive rate\n(also known as sensitivity or recall)")
# plt.xlim(0, 1)
# plt.ylim(0, 1)
# plt.legend(loc="lower right")


In [ ]:
# Boxplot of the mean results of each supervised model for comparison 
names = ['LRC', 'SVC', 'GBC', 'GNB', 'KNC', 'DT','RFC', 'ANN']
fig = plt.figure(figsize=(7,4))
fig.suptitle('Model Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.grid()
plt.savefig('BoxPlot_Curves.png')
plt.show()